<a href="https://colab.research.google.com/github/khoinguyen-hvkn/MaSSP/blob/master/attention/Lecture_note_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lưu ý chung

**Ngôn ngữ**: Để hạn chế các rào cản về ngôn ngữ, bài giảng này sẽ được soạn bằng tiếng Việt, các thuật ngữ chuyên môn sẽ được giữ nguyên bằng tiếng Anh (có dịch tiếng Việt trong ngoặc đi kèm).

**Kiến thức tiên quyết**:
* Nhân ma trận
* Neural networks (mạng neuron nhân tạo) và feed-forward neural networks
* Tensor (i.e. ma trận với số chiều tùy ý)
* Recurrent neural networks

**Thư viện sử dụng**: Keras (ngôn ngữ Python), NumPy và OpenCV (các bạn tham khảo cách cài đặt ở dưới đây)

# Lý thuyết

## Giới thiệu

Attention mechanism (cơ chế chú ý) ban đầu được đề xuất như một phương pháp giải quyết bài toán machine translation (dịch máy). Trong bài toán machine translation, chúng ta phải dịch một câu, từ một ngôn ngữ này, sang một ngôn ngữ khác. 

**Các phương pháp truyền thống**:

Đã có rất nhiều phương pháp được đề xuất trong bài toán machine translation. Ở những giai đoạn đầu tiên, các nhà ngôn ngữ học được mời đến tham gia xử lý bài toán dịch máy, họ giải quyết bài toán bằng những quy luật ngôn ngữ, giống như cách AI thời kì đầu giải quyết vấn đề bằng if ... else vậy. Đến một thời điểm, khi người ta nhận ra "sa thải một nhà ngôn ngữ học giúp độ chính xác của mô hình tăng gấp đôi", các nhà nghiên cứu bắt đầu hướng tới các mô hình xác suất (probabilistic models). Các mô hình xác suất tỏ rõ ưu thế và chiếm vị trí state-of-the-art (phương pháp tốt nhất) trong nhiều thập kỉ. Những năm gần đây, với sự phát triển mạnh mẽ, deep learning đã đánh bật các mô hình xác suất và trở thành state-of-the-art của bài toán machine translation.

**Mô hình encoder-decoder**:

![Encoder-decoder model](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/08/Encoder-Decoder-Recurrent-Neural-Network-Model.png)

Một kiểu kiến trúc kinh điển của deep learning, được đề xuất từ nửa sau của thế kỉ 20, là RNN - một mô hình chuyên xử lý các chuỗi dữ liệu (sequence) có độ dài lớn. RNN ban đầu được áp dụng vào bài toán machine translation theo mô hình encoder-decoder. Cụ thể, encoder là một mạng RNN với đầu ra là một context vector. Nhiệm vụ của encoder là trích xuất context (ngữ cảnh) của câu đầu vào (input sentence hoặc câu cần dịch). Từ context vector, một bộ decoder với cấu trúc RNN sẽ có nhiệm vụ dịch lại input sentence sang ngôn ngữ mới và cho ra một output sentence.

## Attention mechanism

**Ý tưởng**:

Trong cấu trúc encoder-decoder truyền thống, context vector có size cố định, điều này tạo ra một gánh nặng cho mô hình machine translation. Cụ thể, mô hình của chúng ta sẽ phải lưu trữ thông tin của một câu với độ dài có thể lên tới ... vô hạn vào trong một vector có độ dài hữu hạn.

Để khắc phục hạn chế này, các nhà khoa học đề xuất attention mechanism (cơ chế tập trung). Cụ thể, ta vẫn sử dụng một bộ encoder-decoder như thông thường, tuy nhiên context vector của ta sẽ có kích cỡ tùy biến.

**Mô hình**:

![Attention mechanism](https://devblogs.nvidia.com/wp-content/uploads/2015/07/Figure4_attention_2.png)

Trong bài toán machine translation, giả sử ta có một câu (sentence) gồm $T_x$ từ (words) $x_1, ..., x_{T_x}$ trong tiếng Việt và ta muốn dịch câu đó sang tiếng Anh, tức một câu gồm $T_y$ từ $y_1, ..., y_{T_y}$. Công việc của một mô hình là dự đoán $p(y_i|y_1, ..., y_{i-1}, x_1, ..., x_{T_x})$, tức xác suất của $y_i$ khi biết giá trị của $y_1, ..., y_{i-1}$ và $x_1, ..., x_{T_x}$. Ở đây, "xác suất của $y_i$" nghĩa là xác suất để $y_i$ là từng từ trong từ điển, tức là nếu trong từ điển tiếng Anh của chúng ta có $N$ từ thì $p(y_i|y_1, ..., y_{i-1}, x_1, ..., x_{T_x})$ là một vector $N$ chiều với phần tử thứ $j$ thể hiện xác suất để $y_i$ là từ thứ $j$ trong từ điển.

Bộ encoder của chúng ta sẽ là một mạng RNN hai chiều (bi-directional RNN) với các hidden states (trạng thái ẩn) $h_1, ..., h_{T_x}$. Cụ thể, encoder sẽ gồm một forward RNN với hidden states $h^f_1, ..., h^f_{T_x}$ và backward RNN với hidden states $h^b_1, ..., h^b_{T_x}$. Do tính chất của RNN, $h^f_i$ sẽ tổng hợp thông tin của các từ gần bên trái $x_i$, $h^b_i$ sẽ tổng hợp thông tin của các từ gần bên phải $x_i$. Với các hidden states của forward RNN và backward RNN, ta nối chúng lại với nhau theo công thức $h_i = \text{concatenate }(h^f_i, h^b_i)$. Tác dụng của việc nối là hidden state $h_i$ sẽ tổng hợp được thông tin của các từ lân cận $x_i$ (tức context của $x_i$). Như vậy bộ encoder của chúng ta sẽ có một dãy các hidden states $h_1, ..., h_{T_x}$.

Bộ decoder của chúng ta sẽ là một mạng RNN có các hidden states $s_1, ..., s_{T_y}$ với $s_i = f(s_{i-1}, y_{i-1}, c_i)$. $s_{i-1}$ là hidden state liền trước, $y_{i-1}$ là predicted word thứ $i-1$ (i.e. từ thứ $i-1$ trong câu tiếng Anh) và $c_i$ là thành phần thứ $i$ của context vector. Ta có thể tạm hiểu, để có được hidden state thứ $i$ trong decoder, ta cần đầu vào là hidden state liền trước, từ vừa dịch ra từ câu tiếng Việt, và context của từ hiện tại. Sau khi có được hidden state thứ $i$, ta dự đoán xác suất của từ thứ $i$ trong câu dịch (tiếng Anh) bằng công thức $p(y_i|y_1, ..., y_{i-1}, x_1, ..., x_{T_x}) = g(y_{i-1}, s_i, c_i)$ với $g$ là hàm số nào đó tùy vào cơ chế dịch ngôn ngữ của mô hình. Với cơ chế dự đoán xác suất $p(y_i|y_1, ..., y_{i-1}, x_1, ..., x_{T_x})$ như trên thì $g$ có thể là hàm softmax.

Đóng góp chính của attention mechanism chính là context vector $c$. Cụ thể, $c_i = \sum_{j=1}^{T_x} \alpha_{ij} h_j$ với $\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})}$ và $e_{ij} = a(s_{i-1}, h_j)$. $a(\cdot)$ là một hàm số được biểu diễn bằng một mạng feed-forward đơn giản. $e_{ij}$ là một score cho biết mức độ alignment (phù hợp, khớp với nhau) của hidden state $s_{i-1}$ của decoder và hidden state $h_j$ của encoder. Hiểu đơn giản, $e_{ij}$ là mức độ alignment của input word $x_j$ và output word $y_i$. $\alpha_{ij}$ chính là xác suất mà context của $y_i$ được bao chứa trong hidden state $h_j$ của encoder (tức context xung quanh $x_j$ trong câu đầu vào). Sau khi lấy "giá trị trung bình" (theo luật xác suất) của context của $y_i$, ta được context của $y_i$ là $c_i = \sum_{j=1}^{T_x} a_{ij} h_j$. Từ khóa "attention" được thể hiện ở chỗ, $c_i$ không lấy context của toàn bộ câu đầu vào, mà chỉ tập trung vào một số vị trí cụ thể, là các vị trí với trọng số $a_{ij}$ lớn, thực sự hữu ích cho việc dự đoán từ $y_i$ của câu đầu ra.

**Hệ quả**:

Attention mechanism loại bỏ gánh nặng phải trích xuất thông tin của cả input sentence trong hidden state $h_i$ của encoder. Hidden state $h_i$ của encoder vì thế có thể tập trung trích xuất những thông tin của các từ lân cận của $x_i$, hơn là cố gắng nhồi nhét thông tin của cả câu.

Như ta đã biết, RNNs rất kém hiệu quả trong việc để hidden state $h_i$ của encoder tổng hợp được thông tin của các vị trí rất xa $i$ trong câu đầu vào (khoảng 10 - 20 vị trí). Cụ thể, vấn đề này là do gradient explosion và gradient vanishing. Thử tưởng tượng một mô hình RNN như sau:

$\hspace{2.0cm} h_i = f(W h_{i-1})$

Với $h_i$ là hidden state thứ $i$, $f$ là hàm phi tuyến và $W$ là tham số cần học bởi mô hình. Để đơn giản hóa, ta coi $h_i \in \textbf{R}$, $W \in \textbf{R}$ và $f: \textbf{R} \to \textbf{R}$. Ta có quan sát như sau:

$\hspace{2.0cm} \frac{\partial h_i}{\partial h_{i-k}} = \prod_{j=i-k+1}^i \frac{\partial h_i}{\partial f} \frac{\partial f}{\partial (W h_{i-1})} W$

$\hspace{2.0cm} = W^k \prod_{j=i-k+1}^i \frac{\partial h_i}{\partial f} \frac{\partial f}{\partial (W h_{i-1})}$

Như vậy ta có $\frac{\partial h_i}{\partial h_{i-k}} \propto W^k$. Nếu $W < 1$ thì $\frac{\partial h_i}{\partial h_{i-k}} \to 0$ khi $k \to \infty$, mà $\frac{\partial h_i}{\partial W} \propto \frac{\partial h_i}{\partial h_{i-k}}$ nếu chỉ xét riêng tại đạo hàm theo $W$ tại vị trí $i-k$. Như vậy, gradient descent gần như không thay đổi giá trị của tham số $W$ nếu chúng ta đi quá xa khỏi $i$. Xét trường hợp khác, khi $W > 1$, khi đó $\frac{\partial h_i}{\partial h_{i-k}} \to \infty$ khi $k \to \infty$. Điều này dẫn tới việc gradient descent sẽ đưa chúng ta tới một giá trị $W$ nào đó với giá trị của loss function có thể lên tới ... vô cùng. Quy chung lại, ta thấy việc trích xuất thông tin từ các vị trí rất xa $i$ là rất khó đối với một mạng RNN thông thường.

# Thực hành

## Chuẩn bị dữ liệu

In [0]:
import tensorflow as tf
from keras_preprocessing import sequence
from tensorflow import keras
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Concatenate
 
vocab_size = 10000
 
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2
 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size, start_char=start_id,
                                                                        oov_char=oov_id, index_from=index_offset)
 
word2idx = tf.keras.datasets.imdb.get_word_index()
 
idx2word = {v + index_offset: k for k, v in word2idx.items()}
 
idx2word[pad_id] = '<PAD>'
idx2word[start_id] = '<START>'
idx2word[oov_id] = '<OOV>'
 
max_len = 200
rnn_cell_size = 128
 
x_train = sequence.pad_sequences(x_train,
                                 maxlen=max_len,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test, maxlen=max_len,
                                truncating='post',
                                padding='post',
                                value=pad_id)

1646592/1641221 [==============================] - 0s 0us/step


## Khởi tạo attention layer

In [0]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

## Khởi tạo model

In [0]:
import os

# Input layer
sequence_input = Input(shape=(max_len,), dtype='int32')

# Embedding layer
embedded_sequences = keras.layers.Embedding(vocab_size, 128, input_length=max_len)(sequence_input)

# Bidirectional RNN (encoder)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM
                                     (rnn_cell_size,
                                      dropout=0.3,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_activation='relu',
                                      recurrent_initializer='glorot_uniform'), name="bi_lstm_0")(embedded_sequences)
lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional \
    (tf.keras.layers.LSTM
     (rnn_cell_size,
      dropout=0.2,
      return_sequences=True,
      return_state=True,
      recurrent_activation='relu',
      recurrent_initializer='glorot_uniform'))(lstm)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

context_vector, attention_weights = Attention(100)(lstm, state_h)

# Output layer 
output = keras.layers.Dense(1, activation='sigmoid')(context_vector)
 
model = keras.Model(inputs=sequence_input, outputs=output)
 
# Summarize layers
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 200, 128)     1280000     input_8[0][0]                    
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectional)       [(None, 200, 256), ( 263168      embedding_7[0][0]                
__________________________________________________________________________________________________
bidirectional_7 (Bidirectional) [(None, 200, 256), ( 394240      bi_lstm_0[0][0]                  
                                                                 bi_lstm_0[0][1]            

## Huấn luyện model

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=200,
                    validation_split=.3, verbose=1, callbacks=[early_stopping_callback])

## Đánh giá model

In [0]:
result = model.evaluate(x_test, y_test)
print(result)